# Imports

In [1]:
import pandas as pd
import numpy as np
import gc
from warnings import filterwarnings
from sklearn.metrics import mean_squared_error

train_path = r'D:/KaggleTabular/August/train.csv'
test_path = r'D:/KaggleTabular/August/test.csv'
submission_path = r'D:/KaggleTabular/August/sample_submission.csv'

rs = 69420
filterwarnings('ignore')

In [2]:
train = pd.read_csv(train_path, index_col=0)
train.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
id,,,,,,,,,,,,,,,,,,,,,
0,-0.002350,59,0.766739,-1.350460,42.2727,16.68570,30.3599,1.267300,0.392007,1.09101,...,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980,15
1,0.784462,145,-0.463845,-0.530421,27324.9000,3.47545,160.4980,0.828007,3.735860,1.28138,...,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383,3
2,0.317816,19,-0.432571,-0.382644,1383.2600,19.71290,31.1026,-0.515354,34.430800,1.24210,...,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550,6
3,0.210753,17,-0.616454,0.946362,-119.2530,4.08235,185.2570,1.383310,-47.521400,1.09130,...,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.45645,10.02880,2
4,0.439671,20,0.968126,-0.092546,74.3020,12.30650,72.1860,-0.233964,24.399100,1.10151,...,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.78485,7.07197,1


In [3]:
X = train.iloc[:, :-1].values
y = train.iloc[:, -1].values

X.shape, y.shape

((250000, 100), (250000,))

In [4]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()

X = sc.fit_transform(X)

# AutoKeras

In [5]:
import pandas as pd
import tensorflow as tf
import autokeras as ak

In [6]:
clf = ak.StructuredDataRegressor(overwrite=False,
                                 project_name='tps-aug-21',
                                 metrics=[tf.keras.metrics.RootMeanSquaredError()],
                                 tuner='bayesian',
                                 max_trials=100,
                                 objective="val_loss")

INFO:tensorflow:Reloading Oracle from existing project .\tps-aug-21\oracle.json
INFO:tensorflow:Reloading Tuner from .\tps-aug-21\tuner0.json


In [7]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, verbose=1)

In [8]:
%%time
clf.fit(X, y,
        validation_split=0.15,
        batch_size=512,
        epochs=50,
        shuffle=True,
        callbacks=[es],
        verbose=1)

Trial 99 Complete [00h 06m 01s]
val_loss: 62.615787506103516

Best val_loss So Far: 61.98082733154297
Total elapsed time: 03h 19m 27s
INFO:tensorflow:Oracle triggered exit
Epoch 1/50
489/489 [==============================] - 8s 14ms/step - loss: 84.0498 - root_mean_squared_error: 9.0160
Epoch 2/50
489/489 [==============================] - 7s 14ms/step - loss: 68.7547 - root_mean_squared_error: 8.2918
Epoch 3/50
489/489 [==============================] - 7s 14ms/step - loss: 67.1755 - root_mean_squared_error: 8.1961
Epoch 4/50
489/489 [==============================] - 7s 14ms/step - loss: 66.4001 - root_mean_squared_error: 8.1486
Epoch 5/50
489/489 [==============================] - 7s 14ms/step - loss: 65.6234 - root_mean_squared_error: 8.1008
Epoch 6/50
489/489 [==============================] - 7s 14ms/step - loss: 64.9991 - root_mean_squared_error: 8.0622
Epoch 7/50
489/489 [==============================] - 7s 14ms/step - loss: 64.4652 - root_mean_squared_error: 8.0290
Epoch 8/5

In [9]:
model = clf.export_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 100)               0         
_________________________________________________________________
normalization (Normalization (None, 100)               201       
_________________________________________________________________
dense (Dense)                (None, 512)               51712     
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
re_lu (ReLU)                 (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0     

In [ ]:
model.save("autokeras_tps_aug", save_format="tf")